# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK


## Import required libraries and environment variables

In [2]:
# Import required libraries
import os
import json
import openai
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# Configure environment variables
load_dotenv()

True

In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SearchField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)

In [4]:
# Configure environment variables
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY
credential = AzureKeyCredential(key)

## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [5]:
# test embedding with langchain
embeddingmodel = OpenAIEmbeddings(engine=OPENAI_ADA_EMBEDDING_MODEL_NAME, deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,chunk_size=1)
vec = embeddingmodel.embed_query("transform to vec")
vec

c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
  warnings.warn(


[-0.009420825504358491,
 -0.004646901672600355,
 -0.0015674912696747403,
 -0.0068662648674522415,
 0.000535875636165667,
 0.015242684244644191,
 -0.020154216113981882,
 -0.02011187446294001,
 -0.031106366953601014,
 -0.05140171755821695,
 -0.002101161592112756,
 0.005289070496772616,
 -0.01695042909018132,
 3.8068120822798044e-05,
 0.009371428153239788,
 0.004029431989648748,
 0.016357659014111674,
 0.0003096170368160593,
 0.009187951080241254,
 -0.013986574053220046,
 -0.011629603286449911,
 -0.005095008476674919,
 0.005881841504164871,
 -0.012631669159558493,
 -0.020888122543330803,
 0.0045586919174621125,
 0.006936832578430574,
 -0.02297693572898649,
 -0.01630120410027092,
 -0.003579561027724324,
 0.010302925849708735,
 0.0035178138731646416,
 -0.004537521091941179,
 -0.028904645802909038,
 -0.009667814122597219,
 -0.019392081296389977,
 0.005659552027146798,
 -0.011827193622247328,
 0.0056736657556069854,
 0.001552495578704948,
 0.011686058200290663,
 -0.00999242661755242,
 -0.0149

In [6]:
# Generate Document Embeddings using OpenAI Ada 002

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(page):
    response = openai.Embedding.create(input=page, engine=OPENAI_ADA_EMBEDDING_MODEL_NAME)

    embeddings = response['data'][0]['embedding']
    return embeddings

## Prepare data for loading into Azure Cognitive Search - DO THIS ONLY ONCE !!!

In [7]:
doc_title = "Semantic Kernel"
# load pdf and split into pages
fileName = "./data/semantic-kernel.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
print("Number of pages: ", len(pages))

doc_with_vector_list = []
doc_id = 0
# Generate embeddings for title and content fields
for page in pages:
    page_with_vector = {}
    page_with_vector['id'] = str(doc_id)
    page_with_vector['title'] = doc_title
    page_with_vector['titleVector'] = generate_embeddings(doc_title)
    page_with_vector['content'] = page.page_content
    page_with_vector['contentVector'] = generate_embeddings(page.page_content)
    doc_with_vector_list.append(page_with_vector)
    doc_id += 1

# Output embeddings to docVectors.json file
with open("./sk_Vectors.json", "w") as f:
    json.dump(doc_with_vector_list, f)

Number of pages:  187


## Create search index - DO THIS ONLY ONCE !!!
Create your search index schema and vector search configuration:

In [8]:
# Create a search index
# Note: You must create Cognitive Search resource and get the endpoint and key in advance
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)

fields = [
    # doc id - mandatory field
    SimpleField(name="id", type=SearchFieldDataType.String, key=True,sortable=True, filterable=True, facetable=True),

    # title and titleVector
    SearchableField(name="title", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),searchable=True, vector_search_dimensions=1536, vector_search_configuration="sk-vector-config"),

    # content and contentVector
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),searchable=True, vector_search_dimensions=1536, vector_search_configuration="sk-vector-config"),

]

#The Hierarchical Navigable Small World (HNSW) graph algorithm is a popular method for approximate nearest neighbour search in high-dimensional spaces.
vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="sk-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,                  #maximum number of edges per node in the zero or base layer of the HNSW graph.
                "efConstruction": 400,   #this parameter affects the index building during the construction phase.Increasing efConstruction will usually improve the quality of the constructed graph, leading to better recall. However, it will also slow down the index building process.
                "efSearch": 500,         #this parameter affects the search time of the query phase. A higher value of efSearch increases the search time but usually results in better recall. 
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="sk-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name="sk-cogsrch-vector-index-2", fields=fields,vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 sk-cogsrch-vector-index-2 created


## Insert text and embeddings into vector store - DO THIS ONLY ONCE !!
Add texts and metadata from the JSON data to the vector store:

In [9]:
# Upload documents to the index
with open('./sk_Vectors.json', 'r') as file:
    documents = json.load(file)
search_client = SearchClient(endpoint=service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents")

Uploaded 187 documents


## Perform a vector similarity search

In [10]:
# Pure Vector Search
query = "semantic kernel?"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query),
    top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.8773118
Content: Tell us about y our PDF experience.What is Semantic Kernel?
Article •07/11/2023
Semantic K ernel is an open-source SDK that lets you easily combine AI services like
OpenAI , Azure OpenAI , and Hugging F ace  with conventional programming
languages like C# and Python. By doing so, you can create AI apps that combine the
best of both worlds.
During K evin Scott's talk The era of the AI Copilot , he showed how Microsoft powers its
Copilot system  with a stack of AI models and plugins. At the center of this stack is an AI
orchestration layer that allows us to combine AI models and plugins together to create
brand new experiences for users.
Semantic Kernel is at the center of the copilot
stack
Title: Semantic Kernel
Score: 0.8760598
Content: Responsible AI and Semantic Kernel
Article •05/23/2023
An AI system includes not only the technology, but also the people who will use it, the
people who will be affected by it, and the environment in whi

In [11]:

query = "semantic kernel planner and kernel"

search_client = SearchClient(
    service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.87037426
Content: To simplify the creation of AI apps, open source projects like LangChain  have
emerged. Semantic K ernel is Microsoft's contribution to this space and is designed to
support enterprise app developers who want to integrate AI into their existing apps.
By using multiple AI models, plugins, and memory all together within Semantic K ernel,
you can create sophisticated pipelines that allow AI to automate complex tasks for users.
For example, with Semantic K ernel, you could create a pipeline that helps a user send an
email to their marketing team. With memory , you could retrieve information about the
project and then use planner  to autogenerate the remaining steps using available
plugins (e.g., ground the user's ask with Microsoft Graph data, generate a response with
GPT-4, and send the email). Finally, you can display a success message back to your user
in your app using a custom plugin.
Step Component Descr iption
1 Ask It starts with a 

In [12]:
# Pure Vector Search multi-lingual
query = "Planificador semántico del kernel y kernel"

search_client = SearchClient(
    service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.83616716
Content: Responsible AI and Semantic Kernel
Article •05/23/2023
An AI system includes not only the technology, but also the people who will use it, the
people who will be affected by it, and the environment in which it is deployed. Creating
a system that is fit for its intended purpose requires an understanding of how the
technology works, what its capabilities and limitations are, and how to achieve the best
performance. Microsoft’s T ransparency Notes are intended to help you understand how
our AI technology works, the choices system owners can make that influence system
performance and behavior, and the importance of thinking about the whole system,
including the technology, the people, and the environment. Y ou can use T ransparency
Notes when developing or deploying your own system, or share them with the people
who will use or be affected by your system.
Microsoft’s T ransparency Notes are part of a broader effort at Microsoft to put our A

## Perform Cross-Field Vector Search with a filter

In [13]:
# Pure Vector Search with Filter
query = "programming languages supported by semantic kernel"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="titleVector, contentVector",
    filter="title eq 'Semantic Kernel'",
    select=["title", "content"] #searching on two fields title and content
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.01666666753590107
Content: Tell us about y our PDF experience.What is Semantic Kernel?
Article •07/11/2023
Semantic K ernel is an open-source SDK that lets you easily combine AI services like
OpenAI , Azure OpenAI , and Hugging F ace  with conventional programming
languages like C# and Python. By doing so, you can create AI apps that combine the
best of both worlds.
During K evin Scott's talk The era of the AI Copilot , he showed how Microsoft powers its
Copilot system  with a stack of AI models and plugins. At the center of this stack is an AI
orchestration layer that allows us to combine AI models and plugins together to create
brand new experiences for users.
Semantic Kernel is at the center of the copilot
stack
Title: Semantic Kernel
Score: 0.01666666753590107
Content: Supported Semantic Kernel languages
Article •07/18/2023
Semantic K ernel plans on providing support to the following languages:
While the overall architecture of the kernel is consiste

## Perform an Hybrid Search

In [14]:
# Hybrid Search
query = "semantic kernel planner and kernel"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)


results = search_client.search(
    search_text=query,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    filter="title eq 'Semantic Kernel'",
    select=["title", "content",],
    top=3
)

print(type(results))

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}\n")

<class 'azure.search.documents._paging.SearchItemPaged'>
Title: Semantic Kernel
Score: 0.03306011110544205
Content: To simplify the creation of AI apps, open source projects like LangChain  have
emerged. Semantic K ernel is Microsoft's contribution to this space and is designed to
support enterprise app developers who want to integrate AI into their existing apps.
By using multiple AI models, plugins, and memory all together within Semantic K ernel,
you can create sophisticated pipelines that allow AI to automate complex tasks for users.
For example, with Semantic K ernel, you could create a pipeline that helps a user send an
email to their marketing team. With memory , you could retrieve information about the
project and then use planner  to autogenerate the remaining steps using available
plugins (e.g., ground the user's ask with Microsoft Graph data, generate a response with
GPT-4, and send the email). Finally, you can display a success message back to your user
in your app using a 

## Perform a Semantic Hybrid Search

In [16]:
# Semantic Hybrid Search
query = "semantic kernel planner and kernel"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=query,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='sk-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Content: {result['content']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: Semantic Kernel.<em> any other operation that you can do in code that is ill-suited for</em> LLMs<em> (e.g., performing calculations).</em> Instead of providing a separate configuration file with semantic descriptions,<em> planner is able to use annotations in the code to understand how the function behaves.</em>
Semantic Answer Score: 0.71630859375

Title: Semantic Kernel
Content: To instantiate planner, all you need to do is pass it a kernel object. Planner will then
automatically discover all of the plugins registered in the kernel and use them to create
plans. The following code initializes both a kernel and a SequentialPlanner. At the end
of this article we'll review the other types of Planners that are available in Semantic
Kernel.
C#      return (
          Convert.ToDouble(context[ "input"], 
CultureInfo.InvariantCulture) -
          Convert.ToDouble(context[ "number2" ], 
CultureInfo.InvariantCulture)
      ).ToString(CultureInfo.InvariantCulture);
  }
  [SKFu